<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=56655c9303218d202612974e4dd3b44900ca5bde6227a65fa6c3dbe7fa52ae3a
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-28 13:00:05
-------------------
qualified stocks: 89
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  2.00 K
Current:  1.45 C
-------------------
Today PnL: -29.63 K (-0.2%)
Current PnL: -19.33 L (-12.67%)
CY Booked + Current PnL: -7.23 L (-4.74%)
-------------------
Total profit:  2.46 L
Total loss:  -21.79 L
-------------------
Total Booked + Current PnL: 19.75 L (15.72%)
Total Booked PnL: 39.08 L (31.11%)
Curr Year Booked PnL: 12.10 L (8.35%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 87.23 L (60.22%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.49%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,127.53,81.0,M-SC,1.54,97988.0,-2789.0,2881.0,15.62,-2.77,2.94,0.09,245.0,-0.97,0.68,28.73,OX40N,NTT,DURABLES
59,RELIANCE,1533.0,-7.28,73.0,X-LC,3.84,231941.0,21575.0,7213.0,0.18,10.26,3.11,13.68,37.0,2.99,1.60,28.47,XY25,NTT,REFINERIES
80,UJJIVANSFB,60.0,131.04,74.0,H-SC,17.52,146232.0,3753.0,15764.0,0.30,2.63,10.78,13.70,163.0,0.24,1.01,72.59,OX40N,NTT,BANKS
2,ABBOTINDIA,35195.0,-9.24,44.0,X-MC,2.98,89670.0,-1008.0,15916.0,0.40,-1.11,17.75,16.44,101.0,-0.06,0.62,18.84,X40,ATH,PHARMA
42,ITC,452.0,-36.66,64.0,X-LC,1.54,204573.0,4435.0,16448.0,-0.55,2.22,8.04,10.44,4.0,0.27,1.41,7.90,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,127.53,81.0,M-SC,1.54,97988.0,-2789.0,2881.0,15.62,-2.77,2.94,0.09,245.0,-0.97,0.68,28.73,OX40N,NTT,DURABLES
39,INDUSINDBK,1800.0,268.35,66.0,L-MC,9.25,45224.0,-31982.0,57376.0,3.03,-41.42,126.87,32.89,258.0,-0.56,0.31,24.54,XR,NTT,BANKS
84,VALIANTORG,838.0,-290.89,45.0,H-SC,8.68,106097.0,-57508.0,178826.0,2.41,-35.15,168.55,74.15,139.0,-0.32,0.73,36.00,XR,NTT,CHEMICALS
62,SAIL,228.0,45.15,55.0,M-MC,9.79,228905.0,3943.0,163713.0,2.33,1.75,71.52,74.53,192.0,0.02,1.58,34.57,XY24,BTT,STEEL
54,QUESS,986.0,-45.45,51.0,X-SC,38.35,54422.0,-10584.0,161514.0,2.16,-16.28,296.78,232.18,198.0,-0.07,0.38,6.15,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-37.87,36.0,X-SC,4.48,92943.0,-35727.0,81028.0,-3.98,-27.77,87.18,35.20,219.0,-0.44,0.64,6.68,X40,NTT,FOOTWEAR
15,BSOFT,836.99,-14.41,58.0,H-SC,13.97,96058.0,-44595.0,117373.0,-2.59,-31.71,122.19,51.74,131.0,-0.38,0.66,11.14,XR,ATH,IT
60,REPCOHOME,880.00,-56.04,64.0,H-SC,4.02,253380.0,-33125.0,290449.0,-2.24,-11.56,114.63,89.82,134.0,-0.11,1.75,34.06,XY24,NTT,FINANCE
32,HINDZINC,730.22,25.96,45.0,M-LC,11.19,205516.0,440.0,112130.0,-2.04,0.21,54.56,54.89,52.0,0.00,1.42,24.36,X5K,ATH,METALS
86,VOLTAS,1530.00,0.74,51.0,X-MC,3.51,211260.0,19518.0,18232.0,-1.96,10.18,8.63,19.69,99.0,1.07,1.46,17.67,XY25,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.69,64.0,H-SC,16.40,200999.0,1045.0,96178.0,0.67,0.52,47.85,48.63,115.0,0.01,1.39,12.14,XR,NTT,MISC
57,RECLTD,446.00,45.36,49.0,M-LC,5.49,203692.0,522.0,41614.0,-0.90,0.26,20.43,20.74,55.0,0.01,1.41,7.14,XY25,NTT,FINANCE
88,WIPRO,420.00,-14.25,46.0,M-LC,5.78,150344.0,-601.0,110052.0,-0.58,-0.40,73.20,72.51,53.0,-0.01,1.04,5.60,XR,NTT,IT
32,HINDZINC,730.22,25.96,45.0,M-LC,11.19,205516.0,440.0,112130.0,-2.04,0.21,54.56,54.89,52.0,0.00,1.42,24.36,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.0,131.04,74.0,H-SC,17.52,146232.0,3753.0,15764.0,0.30,2.63,10.78,13.70,163.0,0.24,1.01,72.59,OX40N,NTT,BANKS
79,TTKPRESTIG,770.0,127.53,81.0,M-SC,1.54,97988.0,-2789.0,2881.0,15.62,-2.77,2.94,0.09,245.0,-0.97,0.68,28.73,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.61,49.0,H-SC,2.58,223560.0,-46107.0,82449.0,-1.89,-17.10,36.88,13.47,138.0,-0.56,1.54,13.67,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-22.37,54.0,H-MC,7.06,104984.0,-26851.0,69069.0,-1.07,-20.37,65.79,32.03,98.0,-0.39,0.72,17.59,OX40N,NTT,IT
68,SIS,528.0,2054.92,49.0,H-SC,3.14,86968.0,-24064.0,47676.0,-0.28,-21.67,54.82,21.26,156.0,-0.50,0.60,17.50,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.0,-15.53,76.0,H-MC,15.24,216904.0,37096.0,75309.0,0.94,20.63,34.72,62.52,88.0,0.49,1.5,61.75,XR,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.00,131.04,74.0,H-SC,17.52,146232.0,3753.0,15764.0,0.30,2.63,10.78,13.70,163.0,0.24,1.01,72.59,OX40N,NTT,BANKS
57,RECLTD,446.00,45.36,49.0,M-LC,5.49,203692.0,522.0,41614.0,-0.90,0.26,20.43,20.74,55.0,0.01,1.41,7.14,XY25,NTT,FINANCE
62,SAIL,228.00,45.15,55.0,M-MC,9.79,228905.0,3943.0,163713.0,2.33,1.75,71.52,74.53,192.0,0.02,1.58,34.57,XY24,BTT,STEEL
32,HINDZINC,730.22,25.96,45.0,M-LC,11.19,205516.0,440.0,112130.0,-2.04,0.21,54.56,54.89,52.0,0.00,1.42,24.36,X5K,ATH,METALS
47,KPIGREEN,731.05,9.79,69.0,H-SC,5.28,130599.0,5302.0,53624.0,-0.76,4.23,41.06,47.03,141.0,0.10,0.90,63.52,MH,ATH,POWER


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-15.53,76.0,H-MC,15.24,216904.0,37096.0,75309.0,0.94,20.63,34.72,62.52,88.0,0.49,1.50,61.75,XR,NTT,BANKS
80,UJJIVANSFB,60.00,131.04,74.0,H-SC,17.52,146232.0,3753.0,15764.0,0.30,2.63,10.78,13.70,163.0,0.24,1.01,72.59,OX40N,NTT,BANKS
87,WHIRLPOOL,2270.00,-38.52,64.0,M-SC,3.18,127420.0,12622.0,81421.0,-0.45,11.00,63.90,81.92,223.0,0.16,0.88,50.85,XR,NTT,DURABLES
47,KPIGREEN,731.05,9.79,69.0,H-SC,5.28,130599.0,5302.0,53624.0,-0.76,4.23,41.06,47.03,141.0,0.10,0.90,63.52,MH,ATH,POWER
37,INDIAMART,4810.62,-53.15,57.0,H-SC,6.90,130698.0,7362.0,124268.0,0.44,5.97,95.08,106.72,119.0,0.06,0.90,30.74,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-53.17,19.0,X-LC,16.99,148848.0,-127675.0,237740.0,1.67,-46.17,159.72,39.81,54.0,-0.54,1.03,4.91,XY24,NTT,AUTO
24,DMART,5391.45,-17.56,35.0,X-LC,9.99,100848.0,-1738.0,28550.0,-1.31,-1.69,28.31,26.13,38.0,-0.06,0.70,23.44,X40N,ATH,FMCG
45,JSWINFRA,342.00,-20.69,36.0,X-MC,5.38,199091.0,-1256.0,31755.0,-0.35,-0.63,15.95,15.23,178.0,-0.04,1.37,31.42,X40N,NTT,REALTY
13,BATAINDIA,2096.00,-37.87,36.0,X-SC,4.48,92943.0,-35727.0,81028.0,-3.98,-27.77,87.18,35.20,219.0,-0.44,0.64,6.68,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-30.68,37.0,X-MC,2.88,200580.0,-240.0,68037.0,-0.26,-0.12,33.92,33.76,80.0,-0.00,1.39,5.07,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-3.25,45.0,H-LC,1.01,155520.0,-30575.0,78055.0,-1.57,-16.43,50.19,25.51,15.0,-0.39,1.07,14.61,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,64.0,X-LC,1.54,204573.0,4435.0,16448.0,-0.55,2.22,8.04,10.44,4.0,0.27,1.41,7.90,X40,NTT,FMCG
53,PGHH,17907.65,-30.68,37.0,X-MC,2.88,200580.0,-240.0,68037.0,-0.26,-0.12,33.92,33.76,80.0,-0.00,1.39,5.07,X40,ATH,FMCG
29,HAVELLS,2069.16,-6.89,42.0,X-MC,3.08,310842.0,-17149.0,123684.0,-0.83,-5.23,39.79,32.48,92.0,-0.14,2.15,5.60,X40,ATH,ELECTRICAL
86,VOLTAS,1530.00,0.74,51.0,X-MC,3.51,211260.0,19518.0,18232.0,-1.96,10.18,8.63,19.69,99.0,1.07,1.46,17.67,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PAGEIND,51769.93,-27.78,38.0,X-MC,6.88,81540.0,-1970.0,21999.0,-0.52,-2.36,26.98,23.98,82.0,-0.09,0.56,3.32,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-14.90,43.0,X-MC,5.86,179203.0,-21303.0,118453.0,-0.50,-10.62,66.10,48.45,90.0,-0.18,1.24,3.34,X40N,ATH,FINANCE
21,COLPAL,3767.14,-4.54,45.0,X-MC,8.06,222700.0,-40665.0,154019.0,0.47,-15.44,69.16,43.04,84.0,-0.26,1.54,3.39,XY25,ATH,FMCG
3,ACC,3906.00,-37.64,50.0,X-MC,2.70,185310.0,-52641.0,205286.0,-0.52,-22.12,110.78,64.15,174.0,-0.26,1.28,3.95,XY24,BTT,CEMENT
85,VBL,671.64,-19.74,47.0,X-LC,9.53,289205.0,-26637.0,139310.0,-1.32,-8.43,48.17,35.67,5.0,-0.19,2.00,4.25,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-45.45,51.0,X-SC,38.35,54422.0,-10584.0,161514.0,2.16,-16.28,296.78,232.18,198.0,-0.07,0.38,6.15,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.78,38.0,X-MC,6.88,81540.0,-1970.0,21999.0,-0.52,-2.36,26.98,23.98,82.0,-0.09,0.56,3.32,X40,ATH,APPARELS
58,RELAXO,1176.00,-40.27,41.0,X-SC,4.69,81470.0,-63690.0,139615.0,-0.37,-43.88,171.37,52.31,136.0,-0.46,0.56,9.05,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.24,44.0,X-MC,2.98,89670.0,-1008.0,15916.0,0.40,-1.11,17.75,16.44,101.0,-0.06,0.62,18.84,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-37.87,36.0,X-SC,4.48,92943.0,-35727.0,81028.0,-3.98,-27.77,87.18,35.20,219.0,-0.44,0.64,6.68,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-27.72,55.0,X-LC,11.67,287612.0,-58344.0,125054.0,-0.82,-16.86,43.48,19.28,1.0,-0.47,1.99,6.33,X40,ATH,IT
40,INFY,2275.00,-18.16,57.0,X-LC,6.57,318968.0,6080.0,165608.0,-0.47,1.94,51.92,54.87,3.0,0.04,2.20,10.39,X40,BTT,IT
42,ITC,452.00,-36.66,64.0,X-LC,1.54,204573.0,4435.0,16448.0,-0.55,2.22,8.04,10.44,4.0,0.27,1.41,7.90,X40,NTT,FMCG
85,VBL,671.64,-19.74,47.0,X-LC,9.53,289205.0,-26637.0,139310.0,-1.32,-8.43,48.17,35.67,5.0,-0.19,2.00,4.25,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,47.0,H-LC,5.99,248640.0,-12979.0,132202.0,-1.11,-4.96,53.17,45.57,7.0,-0.10,1.72,5.80,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7116.67,63.0,L-SC,7.46,79156.0,-14654.0,91275.0,0.59,-15.62,115.31,81.67,269.0,-0.16,0.55,53.29,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,27.65,81216.0,-32333.0,72388.0,0.21,-28.47,89.13,35.27,268.0,-0.45,0.56,98.78,XR,NTT,HOTELS
50,MASFIN,398.61,-20.05,46.0,H-SC,9.13,90630.0,-7350.0,28956.0,0.23,-7.50,31.95,22.05,152.0,-0.25,0.63,31.74,XR,ATH,FINANCE
79,TTKPRESTIG,770.00,127.53,81.0,M-SC,1.54,97988.0,-2789.0,2881.0,15.62,-2.77,2.94,0.09,245.0,-0.97,0.68,28.73,OX40N,NTT,DURABLES
87,WHIRLPOOL,2270.00,-38.52,64.0,M-SC,3.18,127420.0,12622.0,81421.0,-0.45,11.00,63.90,81.92,223.0,0.16,0.88,50.85,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,-197.73,78.0,M-SC,28.48,168120.0,18900.0,125283.0,-0.97,12.67,74.52,96.62,208.0,0.15,1.16,81.94,XY25,NTT,FINANCE
69,SONACOMS,806.63,-31.54,73.0,M-SC,8.28,85190.0,-15969.0,55970.0,0.61,-15.79,65.70,39.54,202.0,-0.29,0.59,20.13,AR,ATH,AUTO
5,ANGELONE,3033.00,17.35,67.0,X-SC,5.33,207493.0,16487.0,41208.0,-1.81,8.63,19.86,30.21,157.0,0.40,1.43,31.31,X40N,NTT,FINANCE
12,BANKINDIA,190.00,-15.53,76.0,H-MC,15.24,216904.0,37096.0,75309.0,0.94,20.63,34.72,62.52,88.0,0.49,1.50,61.75,XR,NTT,BANKS
80,UJJIVANSFB,60.00,131.04,74.0,H-SC,17.52,146232.0,3753.0,15764.0,0.30,2.63,10.78,13.70,163.0,0.24,1.01,72.59,OX40N,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.33
1,25,43.41
2,50,74.08


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.85
LC    31.86
MC    24.27
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.81
X40      16.11
XY25     11.96
XR       11.30
X40N     10.42
AR        7.72
OX40N     7.53
X200      1.74
X5K       1.42
SR        1.07
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.23
X-LC    21.31
X-MC    18.54
M-SC    12.64
M-LC     5.02
X-SC     4.54
H-LC     4.53
H-MC     3.84
M-MC     1.58
L-SC     1.44
L-LC     1.00
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.27,-4.34,38.40
IT,12.35,-17.46,80.45
FINANCE,12.02,-8.59,57.82
MISC,6.85,-15.89,80.10
BANKS,6.66,-3.38,57.76
ELECTRICAL,5.70,-9.45,48.25
PAINTS,5.45,-16.93,34.36
INSURANCE,3.83,0.22,36.98
AUTO,2.75,-44.28,104.50


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3254803.0,22
XR,1313294.0,14
AR,1170569.0,9
X40,824286.0,12
X40N,649178.0,9
OX40N,555068.0,10
XY25,451236.0,8
SR,266266.0,2
X5K,112130.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3337546.0,25
M-SC,1215382.0,15
X-MC,1194086.0,14
X-LC,1052418.0,13
X-SC,521809.0,6
H-MC,322786.0,3
M-LC,282777.0,4
H-LC,282723.0,3
L-SC,253499.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1144794.0      6
           AR         837300.0      5
           XR         777284.0      7
M-SC       XY24       699592.0      6
X-MC       XY24       581466.0      4
X-LC       X40        453097.0      5
           XY24       306335.0      2
X-MC       X40        290161.0      6
X-SC       X40N       279267.0      4
H-SC       SR         266266.0      2
           OX40N      258278.0      4
X-LC       X40N       219703.0      3
H-LC       AR         210257.0      2
H-MC       XY24       178408.0      1
X-MC       XY25       172251.0      2
M-MC       XY24       163713.0      1
L-SC       XR         163663.0      2
X-SC       XY24       161514.0      1
X-MC       X40N       150208.0      2
M-SC       OX40N      137885.0      4
           XR         129610.0      2
           XY25       125283.0      1
           AR         123012.0      2
M-LC       X5K        112130.0      1
           XR         110052.0      1
L-SC       OX40N       89836.0      1
X-SC       X40         81028.0      1
H-MC       XR          75309.0      1
X-LC       XY25        73283.0      3
H-LC       X200        72466.0      1
H-MC       OX40N       69069.0      1
L-MC       XR          57376.0      1
H-SC       MH          53624.0      1
M-LC       XY25        41614.0      1
L-LC       XY25        38805.0      1
M-LC       XY24        18981.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
